In [3]:
import numpy as np
import statsmodels.api as sm
from scipy import stats

# Simulación de Monte Carlo: Evaluación del Estimador MCO y del Estadístico t 


El análisis de Monte Carlo simula una gran cantidad de muestras del modelo para estudiar la
distribución en muestras finitas de los estimadores. En este ejercicio, utilizamos esta técnica para
confirmar dos resultados en muestras finitas presentados en clases: la insesgadez del estimador de
coeficientes MCO y la distribución del cociente t. El modelo es el siguiente modelo de regresión
simple que satisface las suposiciones estudiadas en clase con n = 32. La ecuación de regresión es:
$$y_i = β_1 + β_2x_i + ε_i \quad (i = 1, 2, . . . , n)$$
donde X = (1 : x) y β = $(β_1, β_2)'$. Los parámetros del modelo son $(β_1, β_2, σ_2)$. Elegimos una
distribución particular al especificar el modelo de regresión como sigue: fijamos $β_1 = 1, \quad β_2 = 0,5$
y $σ_2 = 1$. La distribución de x = $(x_1, x_2, . . . , x_n)'$ se especifica mediante el siguiente proceso
autorregresivo de orden 1 AR(1):
$$x_i = c + \phi x_{i−1} + η_i \quad (i = 1, 2, . . . , n)$$
donde ${η_i}$ son variables aleatorias i.i.d. N(0, 1) y 
$$
x_0 \sim \mathcal{N} \left( \frac{c}{1 - \phi}, \frac{1}{1 - \phi^2} \right), \quad c = 2,\ \phi = 0.6
$$
Esto fija la distribución conjunta de (y, X). A partir de esta distribución se generarán muchas
muestras. Para programar la simulación, la siguiente expresión de x será útil. Se resuelve la ecuación
en diferencias de primer orden anterior:
$$
x_i = \phi^i x_0 + \left(1 + \phi + \phi^2 + \cdots + \phi^{i-1} \right) c + \eta_i + \phi \eta_{i-1} + \phi^2 \eta_{i-2} + \cdots + \phi^{i-1} \eta_1
$$
o en notación matricial
$$
\mathbf{x}_{n \times 1} = \mathbf{r}_{n \times 1} \cdot x_0 + \mathbf{d}_{n \times 1} + \mathbf{A}_{n \times n} \boldsymbol{\eta}_{n \times 1}
$$
donde:
$$
\mathbf{d} = (d_1, d_2, \ldots, d_n)', \quad d_i = \left(1 + \phi + \cdots + \phi^{i-1}\right) c
$$

$$
\mathbf{r} =
\begin{bmatrix}
\phi \\
\phi^2 \\
\vdots \\
\phi^n
\end{bmatrix}
\quad
\mathbf{A} =
\begin{bmatrix}
1 & 0 & \cdots & 0 \\
\phi & 1 & \cdots & 0 \\
\phi^2 & \phi & 1 & \cdots \\
\vdots & \vdots & \vdots & \vdots \\
\phi^{n-1} & \phi^{n-2} & \cdots & 1
\end{bmatrix}
\quad
\boldsymbol{\eta} =
\begin{bmatrix}
\eta_1 \\
\eta_2 \\
\vdots \\
\eta_n
\end{bmatrix}
$$



### 1. Inicializar un contador 

In [ ]:
contador=0

### 2. Repetir por muchas iteraciones:
a) Generar una nueva x (nuevo $x_0$ y nueva η).


In [41]:
n = 32
phi = 0.6
c = 2
beta_1 = 1
beta_2 = 0.5
sigma_2 = 1
iteraciones = 10000

In [ ]:
for _ in range(iteraciones):
    ##generamos x y eta
    x0=np.random.normal(loc=c/(1-phi),scale=np.sqrt(c/(1-phi)**2))
    eta=np.random.normal(0,1,n)

    r=phi**np.arange(1,n+1)
    d=np.array([sum([phi**j for j in range(i)])*c for i in range(1,n+1)])
    A=np.zeros((n,n))
    for i in range(n):
        for j in range(i+1):
            A[i,j]=phi ** i-j

    
    x=r*x0+d+A*eta
    

    ##generamo epsilon y y
    epsilon = np.random.normal(0, np.sqrt(sigma_2), n)
    y = beta_1 + beta_2 * x + epsilon





b) Generar ε ∼ N(0, I) y calcular y.
c) Estimar el modelo y calcular el estadístico t para H0 : β2 = 0,5.
d) Si |t| > t0,025(n − 2), incrementar el contad